In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class ImageClassificationDataset(Dataset):
    def __init__(self, positive_dir, negative_dir, transform=None):
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.positive_files = [(os.path.join(positive_dir, f), 1) for f in os.listdir(positive_dir) if f.endswith('.png')]
        self.negative_files = [(os.path.join(negative_dir, f), 0) for f in os.listdir(negative_dir) if f.endswith('.png')]
        self.all_files = self.positive_files + self.negative_files

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self, idx):
        file_path, label = self.all_files[idx]
        image = Image.open(file_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label, os.path.basename(file_path)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

positive_dir = "data/train_images/Train/Positive"
negative_dir = "data/train_images/Train/Negative"
dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

/home/johann/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)  # Output: 2 classes (positive and negative)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()

In [3]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels, _ in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

print("Training complete.")

Epoch [1/10], Loss: 0.3466
Epoch [2/10], Loss: 0.2129
Epoch [3/10], Loss: 0.1981
Epoch [4/10], Loss: 0.1661
Epoch [5/10], Loss: 0.1510
Epoch [6/10], Loss: 0.1764
Epoch [7/10], Loss: 0.1322
Epoch [8/10], Loss: 0.1040
Epoch [9/10], Loss: 0.0789
Epoch [10/10], Loss: 0.0565
Training complete.


In [ ]:
# Function to evaluate the model and output file names
def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    all_filenames = []

    with torch.no_grad():
        for images, labels, filenames in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.numpy())
            all_labels.extend(labels.numpy())
            all_filenames.extend(filenames)

    return all_predictions, all_labels, all_filenames

# Evaluate the model
test_dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions, labels, filenames = evaluate_model(model, test_dataloader)

# Print the results
idx = 0
for filename, prediction, label in zip(filenames, predictions, labels):
    print(f"File: {filename}, Prediction: {prediction}, Label: {label}")
    # print the original image
    image = Image.open(os.path.join(positive_dir, filename))
    image.show()
    idx += 1
    if idx == 5:
        break

File: mask_20241202-163759-647.png, Prediction: 1, Label: 1
File: mask_20241126-162109-143.png, Prediction: 1, Label: 1
File: mask_20241203-170422-829.png, Prediction: 1, Label: 1
File: mask_20241203-083122-181.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241204-092652-029.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241203-084721-591.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241203-164446-728.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241203-163802-444.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241203-084300-162.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241203-160929-199.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241203-161443-013.png, Prediction: 1, Label: 1
Opening in existing browser session.
File: mask_20241203-171123-730.png, Predicti

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_images/Train/Positive/mask_20241203-171527-321.png'